In [15]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

In [16]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1 
min_mem_size=6 
run_time=500
classification=False
scale=False
max_models=9    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0

In [17]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

4


In [18]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions


In [19]:
all_variables=None

In [20]:
data_path='/Users/damik/Downloads/Machine Learning Repo/Final Project/INFO6105-FinalProject/hour.csv'


In [21]:
run_id=alphabet(5)
if server_path==None:
  server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id)

2b59q


In [22]:
logfile=run_id+'_autoh2o_log.zip'
logs_path=os.path.join(run_dir,'logs')
print(logs_path,' ',logfile)

C:\Users\damik\Downloads\Machine Learning Repo\Final Project\INFO6105-FinalProject\MetaData\2b59q\logs   2b59q_autoh2o_log.zip


In [23]:
# 65535 Highest port no
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:25896 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from C:\Users\damik\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\damik\AppData\Local\Temp\tmpsw6_iwu9
  JVM stdout: C:\Users\damik\AppData\Local\Temp\tmpsw6_iwu9\h2o_damik_started_from_python.out
  JVM stderr: C:\Users\damik\AppData\Local\Temp\tmpsw6_iwu9\h2o_damik_started_from_python.err
  Server is running at http://127.0.0.1:25896
Connecting to H2O server at http://127.0.0.1:25896 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_damik_hr0rc9
H2O cluster total nodes:,1
H2O cluster free memory:,3.833 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [24]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,test_path,model_path,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)


{'start_time': 1555615683.23735, 'target': None, 'server_path': 'C:\\Users\\damik\\Downloads\\Machine Learning Repo\\Final Project\\INFO6105-FinalProject\\MetaData', 'data_path': '/Users/damik/Downloads/Machine Learning Repo/Final Project/INFO6105-FinalProject/hour.csv', 'test_path': None, 'max_models': 9, 'run_time': 500, 'run_id': '2b59q', 'scale': False, 'classification': False, 'model_path': None, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1555615683.23735, 'execution_time': 0.0, 'run_path': 'C:\\Users\\damik\\Downloads\\Machine Learning Repo\\Final Project\\INFO6105-FinalProject\\MetaData\\2b59q', 'nthreads': 1, 'min_mem_size': 4, 'analysis': 0}


In [25]:
print(data_path)

/Users/damik/Downloads/Machine Learning Repo/Final Project/INFO6105-FinalProject/hour.csv


In [26]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [27]:
df.head()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011-01-01 00:00:00,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
2,2011-01-01 00:00:00,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40
3,2011-01-01 00:00:00,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32
4,2011-01-01 00:00:00,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
5,2011-01-01 00:00:00,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1
6,2011-01-01 00:00:00,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
7,2011-01-01 00:00:00,1,0,1,6,0,6,0,1,0.22,0.2727,0.8,0,2,0,2
8,2011-01-01 00:00:00,1,0,1,7,0,6,0,1,0.2,0.2576,0.86,0,1,2,3
9,2011-01-01 00:00:00,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7,8
10,2011-01-01 00:00:00,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0,8,6,14


In [28]:
df.describe()

Rows:17379
Cols:17




,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
type,int,time,int,int,int,int,int,int,int,int,real,real,real,real,int,int,int
mins,1.0,1293840000000.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,0.0,1.0
mean,8690.0,1325477314552.046,2.5016399102364923,0.5025605615973301,6.537775476149376,11.546751826917545,0.028770355026181024,3.003682605443351,0.6827205247712756,1.4252833879970077,0.4969871684216583,0.4757751021347602,0.6272288394038783,0.1900976063064618,35.67621842453534,153.78686920996606,189.46308763450145
maxs,17379.0,1356912000000.0,4.0,1.0,12.0,23.0,1.0,6.0,1.0,4.0,1.0,1.0,1.0,0.8507,367.0,886.0,977.0
sigma,5017.029499614288,18150225217.779423,1.1069181394480763,0.5000078290910197,3.438775713750165,6.914405095264493,0.16716527638437123,2.0057714561109883,0.4654306335238829,0.6393568777542534,0.1925561212497219,0.1718502156353594,0.19292983406291508,0.1223402285727905,49.30503038705308,151.35728591258314,181.38759909186476
zeros,0,0,0,8645,0,726,16879,2502,5514,0,0,2,22,2180,1581,24,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,2011-01-01 00:00:00,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01 00:00:00,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,8.0,32.0,40.0
2,3.0,2011-01-01 00:00:00,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,5.0,27.0,32.0


In [29]:
'''
import sys
sys.stdout = open('describe.txt', 'w')
print ('test')
'''

"\nimport sys\nsys.stdout = open('describe.txt', 'w')\nprint ('test')\n"

In [30]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
  target=df.columns[5]   
y = target

In [31]:
print(y)

hr


In [32]:
print(all_variables)

None


In [33]:
if all_variables is not None:
  ivd=get_all_variables_csv(all_variables)
  print(ivd)    
  X=check_all_variables(df,ivd,y)
  print(X)

In [34]:
# independent variables

X = []  
if all_variables is None:
  X=get_independent_variables(df, target)  
else: 
  ivd=get_all_variables_csv(all_variables)    
  X=check_all_variables(df, ivd)


X=check_X(X,df)


# Add independent variables

meta_data['X']=X  


# impute missing values

_=impute_missing_values(df,X, scale)

In [35]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [36]:
# Force target to be factors
# Only 'int' or 'string' are allowed for asfactor(), got Target (Total orders):real 

if classification:
    df[y] = df[y].asfactor()

In [37]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok

In [38]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [39]:
print(ok)

True


In [40]:
classification=True
if classification:
    print(df[y].levels())

[]


In [41]:
print(df[y].levels())

[]


In [42]:
allV=get_variables_types(df)
allV

{'instant': 'int',
 'dteday': 'time',
 'season': 'int',
 'yr': 'int',
 'mnth': 'int',
 'hr': 'int',
 'holiday': 'int',
 'weekday': 'int',
 'workingday': 'int',
 'weathersit': 'int',
 'temp': 'real',
 'atemp': 'real',
 'hum': 'real',
 'windspeed': 'real',
 'casual': 'int',
 'registered': 'int',
 'cnt': 'int'}

In [43]:
meta_data['variables']=allV

In [44]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [45]:
model_start_time = time.time()

In [46]:
aml.train(x=X,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [47]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [48]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [49]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,StackedEnsemble_AllModels_AutoML_20190418_153450,16.575708,4.071328,16.575708,2.871407,NaN
1,StackedEnsemble_BestOfFamily_AutoML_20190418_1...,16.867162,4.106965,16.867162,2.911271,NaN
2,GBM_4_AutoML_20190418_153450,17.321940,4.161963,17.321940,2.944226,NaN
3,GBM_grid_1_AutoML_20190418_153450_model_11,17.779478,4.216572,17.779478,2.956645,NaN
4,XRT_1_AutoML_20190418_153450,17.832009,4.222796,17.832009,3.045021,0.529019
5,GBM_grid_1_AutoML_20190418_153450_model_8,17.892236,4.229922,17.892236,3.035620,NaN
6,GBM_3_AutoML_20190418_153450,18.066172,4.250432,18.066172,3.061447,NaN
7,DRF_1_AutoML_20190418_153450,18.279676,4.275474,18.279676,3.034948,0.528345
8,GBM_grid_1_AutoML_20190418_153450_model_2,18.444320,4.294685,18.444320,3.035934,NaN
9,GBM_2_AutoML_20190418_153450,18.691352,4.323350,18.691352,3.153222,NaN


In [50]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [51]:
mod_best._id# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

'StackedEnsemble_AllModels_AutoML_20190418_153450'

In [52]:
# Get stacked ensemble  
se=get_stacked_ensemble(model_set)

In [53]:
print(se)

StackedEnsemble_BestOfFamily_AutoML_20190418_153450


In [54]:
if se is not None:
  mod_best=h2o.get_model(se)

In [55]:
dir(mod_best)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_end_time',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_make_model',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_run_time',
 '_start_time',
 '_train',
 '_verify_training_frame_params',
 '_xval_keys',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'base_models',
 'biases',
 'blending_frame',
 'catoffsets',
 'coef',
 'coef_norm',
 'convert_H2OXGBoostParams_2_XGBoostParams',

In [56]:
mod_best._id

'StackedEnsemble_BestOfFamily_AutoML_20190418_153450'

In [57]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics(o, train, valid, xval)>

In [58]:
type(mod_best)

h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [59]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190418_153450
No model summary for this model


ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 3.6301960360615575
RMSE: 1.9053073337552546
MAE: 1.3956491037975256
RMSLE: NaN
R^2: 0.9240644000591742
Mean Residual Deviance: 3.6301960360615575
Null degrees of freedom: 17378
Residual degrees of freedom: 17375
Null deviance: 830824.7641406283
Residual deviance: 63089.17691071381
AIC: 71735.97824908436

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 16.86716184492017
RMSE: 4.106965040625519
MAE: 2.911271301064251
RMSLE: NaN
R^2: 0.6471766149073883
Mean Residual Deviance: 16.86716184492017
Null degrees of freedom: 17378
Residual degrees of freedom: 17375
Null deviance: 830873.0196888775
Residual deviance: 293134.4057028676
AIC: 98431.54722881642
<bound method ModelBase.coef_norm of >


In [60]:
'''
mod_best.aic()
 mod_best.algo
 mod_best.coef()
 mod_best.coef_norm()
 mod_best.confusion_matrix()
 mod_best.cross_validation_fold_assignment()
 mod_best.cross_validation_holdout_predictions()
 mod_best.cross_validation_metrics_summary()
 mod_best.cross_validation_models()
 mod_best.cross_validation_predictions()
 mod_best.deepfeatures()
 mod_best.default_params()
 mod_best.download_mojo()
 mod_best.download_pojo()
 mod_best.fit()
 mod_best.full_parameters()
 mod_best.get_params()
 mod_best.get_xval_models()
 mod_best.gini()
 mod_best.have_mojo()
 mod_best.have_pojo()
 mod_best.hit_ratio_table()
 mod_best.is_cross_validated()
 mod_best.join()
 mod_best.keep_levelone_frame()
 mod_best.levelone_frame_id()
 mod_best.logloss()
 mod_best.mae()
 mod_best.mean_per_class_error()
 mod_best.mean_residual_deviance()
 mod_best.metalearner()
 mod_best.metalearner_algorithm()
 mod_best.metalearner_fold_assignment()
 mod_best.metalearner_fold_column()
 mod_best.metalearner_nfolds()
 mod_best.metalearner_params()
 mod_best.mixin()
 mod_best.model_id()
 mod_best.model_performance()
 mod_best.mse()
 mod_best.normmul()
 mod_best.normsub()
 mod_best.null_degrees_of_freedom()
 mod_best.null_deviance()
 mod_best.params()
 mod_best.parms()
 mod_best.partial_plot()
 mod_best.plot()
 mod_best.pprint_coef()
 mod_best.predict()
 mod_best.predict_leaf_node_assignment()
 mod_best.r2()
 mod_best.residual_degrees_of_freedom()
 mod_best.residual_deviance()
 mod_best.respmul()
 mod_best.response_column()
 mod_best.respsub()
 mod_best.rmse()
 mod_best.rmsle()
 mod_best.rotation()
 mod_best.save_model_details()
 mod_best.save_mojo()
 mod_best.score_history()
 mod_best.scoring_history()
 mod_best.seed()
 mod_best.set_params()
 mod_best.show()
 mod_best.start()
 mod_best.std_coef_plot()
 mod_best.summary()
 mod_best.train()
 mod_best.training_frame()
 mod_best.type()
 mod_best.validation_frame()
 mod_best.varimp()
 mod_best.varimp_plot()
 mod_best.weights()
 mod_best.xval_keys()
 mod_best.xvalsmod_best()
 '''


'\nmod_best.aic()\n mod_best.algo\n mod_best.coef()\n mod_best.coef_norm()\n mod_best.confusion_matrix()\n mod_best.cross_validation_fold_assignment()\n mod_best.cross_validation_holdout_predictions()\n mod_best.cross_validation_metrics_summary()\n mod_best.cross_validation_models()\n mod_best.cross_validation_predictions()\n mod_best.deepfeatures()\n mod_best.default_params()\n mod_best.download_mojo()\n mod_best.download_pojo()\n mod_best.fit()\n mod_best.full_parameters()\n mod_best.get_params()\n mod_best.get_xval_models()\n mod_best.gini()\n mod_best.have_mojo()\n mod_best.have_pojo()\n mod_best.hit_ratio_table()\n mod_best.is_cross_validated()\n mod_best.join()\n mod_best.keep_levelone_frame()\n mod_best.levelone_frame_id()\n mod_best.logloss()\n mod_best.mae()\n mod_best.mean_per_class_error()\n mod_best.mean_residual_deviance()\n mod_best.metalearner()\n mod_best.metalearner_algorithm()\n mod_best.metalearner_fold_assignment()\n mod_best.metalearner_fold_column()\n mod_best.met

In [61]:
bm=stackedensemble_df(aml_leaderboard_df)

In [62]:
bm

['GBM_4_AutoML_20190418_153450',
 'GLM_grid_1_AutoML_20190418_153450_model_1',
 'DRF_1_AutoML_20190418_153450',
 'XRT_1_AutoML_20190418_153450',
 'DeepLearning_1_AutoML_20190418_153450']

In [63]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,StackedEnsemble_AllModels_AutoML_20190418_153450,16.575708,4.071328,16.575708,2.871407,NaN
1,StackedEnsemble_BestOfFamily_AutoML_20190418_1...,16.867162,4.106965,16.867162,2.911271,NaN
2,GBM_4_AutoML_20190418_153450,17.321940,4.161963,17.321940,2.944226,NaN
3,GBM_grid_1_AutoML_20190418_153450_model_11,17.779478,4.216572,17.779478,2.956645,NaN
4,XRT_1_AutoML_20190418_153450,17.832009,4.222796,17.832009,3.045021,0.529019
5,GBM_grid_1_AutoML_20190418_153450_model_8,17.892236,4.229922,17.892236,3.035620,NaN
6,GBM_3_AutoML_20190418_153450,18.066172,4.250432,18.066172,3.061447,NaN
7,DRF_1_AutoML_20190418_153450,18.279676,4.275474,18.279676,3.034948,0.528345
8,GBM_grid_1_AutoML_20190418_153450_model_2,18.444320,4.294685,18.444320,3.035934,NaN
9,GBM_2_AutoML_20190418_153450,18.691352,4.323350,18.691352,3.153222,NaN


In [64]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [65]:
bm

['GBM_4_AutoML_20190418_153450',
 'XRT_1_AutoML_20190418_153450',
 'DRF_1_AutoML_20190418_153450',
 'DeepLearning_1_AutoML_20190418_153450',
 'GLM_grid_1_AutoML_20190418_153450_model_1',
 'StackedEnsemble_BestOfFamily_AutoML_20190418_153450']

In [66]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]

In [67]:
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
1,StackedEnsemble_BestOfFamily_AutoML_20190418_1...,16.867162,4.106965,16.867162,2.911271,NaN
2,GBM_4_AutoML_20190418_153450,17.321940,4.161963,17.321940,2.944226,NaN
4,XRT_1_AutoML_20190418_153450,17.832009,4.222796,17.832009,3.045021,0.529019
7,DRF_1_AutoML_20190418_153450,18.279676,4.275474,18.279676,3.034948,0.528345
15,DeepLearning_1_AutoML_20190418_153450,24.536402,4.953423,24.536402,3.734252,0.588782
30,GLM_grid_1_AutoML_20190418_153450_model_1,37.770325,6.145757,37.770325,4.908914,0.740319


In [68]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [69]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)


StackedEnsemble_BestOfFamily_AutoML_20190418_153450


In [70]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

StackedEnsemble_BestOfFamily_AutoML_20190418_153450
stackedensemble


In [71]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [72]:
meta_data['models']=bm

In [73]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass

In [74]:
print(models_path)

C:\Users\damik\Downloads\Machine Learning Repo\Final Project\INFO6105-FinalProject\MetaData\2b59q\models


In [75]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_4_AutoML_20190418_153450', 'varimp': [('registered', 1000072.6875, 1.0, 0.27338784773288016), ('cnt', 638948.5625, 0.6389021223019852, 0.1746680761281091), ('casual', 427871.625, 0.4278405263417415, 0.11696640066947134), ('hum', 258713.046875, 0.2586942430372092, 0.07072386232482926), ('dteday', 190247.40625, 0.1902335786467521, 0.05200754863275915), ('temp', 188861.59375, 0.18884786687067684, 0.05162871187271983), ('atemp', 156216.390625, 0.15620503647141148, 0.04270455872595524), ('weekday', 147985.546875, 0.14797479095738228, 0.040454509618561604), ('windspeed', 144300.59375, 0.14429010566294462, 0.0394471614363423), ('instant', 138248.0625, 0.1382380143243338, 0.03779259321099668), ('mnth', 103349.671875, 0.10334216019173106, 0.028252490754883883), ('weathersit', 103226.6953125, 0.10321919256744025, 0.02821887290073819), ('workingday', 70806.6875, 0.07080154111298036, 0.01935628094104873), ('season', 62980.34375, 0.0629757661989944, 0.01721680918047

In [76]:
    if mod is not None:
        try:    
            sh_df=mod.scoring_history()
            sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
        except:
            pass 
        try:
            stats_dl={}
            stats_dl=dl_stats(mod)
            n=run_id+'_dl_stats.json'
            dict_to_json(stats_dl,n)
            print(stats_dl)
        except:
            pass    
        try:
            cf=mod.confusion_matrix()    
            cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
        except:
            pass

{'algo': 'gbm', 'model_id': 'GBM_4_AutoML_20190418_153450', 'varimp': [('registered', 1000072.6875, 1.0, 0.27338784773288016), ('cnt', 638948.5625, 0.6389021223019852, 0.1746680761281091), ('casual', 427871.625, 0.4278405263417415, 0.11696640066947134), ('hum', 258713.046875, 0.2586942430372092, 0.07072386232482926), ('dteday', 190247.40625, 0.1902335786467521, 0.05200754863275915), ('temp', 188861.59375, 0.18884786687067684, 0.05162871187271983), ('atemp', 156216.390625, 0.15620503647141148, 0.04270455872595524), ('weekday', 147985.546875, 0.14797479095738228, 0.040454509618561604), ('windspeed', 144300.59375, 0.14429010566294462, 0.0394471614363423), ('instant', 138248.0625, 0.1382380143243338, 0.03779259321099668), ('mnth', 103349.671875, 0.10334216019173106, 0.028252490754883883), ('weathersit', 103226.6953125, 0.10321919256744025, 0.02821887290073819), ('workingday', 70806.6875, 0.07080154111298036, 0.01935628094104873), ('season', 62980.34375, 0.0629757661989944, 0.01721680918047

In [77]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass

In [78]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass

In [79]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass

{'algo': 'glm', 'model_id': 'GLM_grid_1_AutoML_20190418_153450_model_1', 'coef': {'Intercept': 352.10303433754126, 'instant': 0.0013746097720389752, 'dteday': -2.623015458674756e-10, 'season': -0.5117192063681029, 'yr': -5.284455207097917, 'mnth': -0.20556048998059095, 'holiday': 0.3219973589098315, 'weekday': -0.07352417309254684, 'workingday': -0.07189931001375928, 'weathersit': 1.2211883124271194, 'temp': -4.336173993347636, 'atemp': 4.996580313969567, 'hum': -7.046786031348421, 'windspeed': 2.2766309683873165, 'casual': 0.008986657058625456, 'registered': 0.008034769577901014, 'cnt': 0.006237236894980357}, 'coef_norm': {'Intercept': 11.546751826917545, 'instant': 6.896457776777611, 'dteday': -4.760832132466382, 'season': -0.5664312718328266, 'yr': -2.642268976029764, 'mnth': -0.7068764206518403, 'holiday': 0.053826777497199566, 'weekday': -0.14747268772319402, 'workingday': -0.033464141409634045, 'weathersit': 0.7807751465833888, 'temp': -0.8349568452229381, 'atemp': 0.858663404395

In [80]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.8])

In [81]:
predictions_df=predictions_test(mod_best,test,run_id)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [82]:
predictions_df.head()

predict
14.0112
19.2145
19.4679
16.1104
18.8292
1.86735
13.1164
16.7841
9.11017
12.215


In [83]:
predictions_df.describe()

Rows:3550
Cols:1




,predict
type,real
mins,-0.9868423783315381
mean,11.557843984161305
maxs,24.292976191144653
sigma,6.218116809243477
zeros,0
missing,0
0,14.011181153109503
1,19.21447893388556
2,19.46794498997935


In [84]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [85]:
meta_data

{'start_time': 1555615683.23735,
 'target': None,
 'server_path': 'C:\\Users\\damik\\Downloads\\Machine Learning Repo\\Final Project\\INFO6105-FinalProject\\MetaData',
 'data_path': '/Users/damik/Downloads/Machine Learning Repo/Final Project/INFO6105-FinalProject/hour.csv',
 'test_path': None,
 'max_models': 9,
 'run_time': 500,
 'run_id': '2b59q',
 'scale': False,
 'classification': False,
 'model_path': None,
 'balance': False,
 'balance_threshold': 0.2,
 'project': None,
 'end_time': 1555617387.6609664,
 'execution_time': 1704.4236164093018,
 'run_path': 'C:\\Users\\damik\\Downloads\\Machine Learning Repo\\Final Project\\INFO6105-FinalProject\\MetaData\\2b59q',
 'nthreads': 1,
 'min_mem_size': 4,
 'analysis': 0,
 'X': ['instant',
  'season',
  'yr',
  'mnth',
  'holiday',
  'weekday',
  'workingday',
  'weathersit',
  'casual',
  'registered',
  'cnt',
  'dteday',
  'temp',
  'atemp',
  'hum',
  'windspeed'],
 'variables': {'instant': 'int',
  'dteday': 'time',
  'season': 'int',
  

In [86]:
# Save logs
h2o.download_all_logs(dirname=logs_path, filename=logfile)

Writing H2O logs to C:\Users\damik\Downloads\Machine Learning Repo\Final Project\INFO6105-FinalProject\MetaData\2b59q\logs\2b59q_autoh2o_log.zip


'C:\\Users\\damik\\Downloads\\Machine Learning Repo\\Final Project\\INFO6105-FinalProject\\MetaData\\2b59q\\logs\\2b59q_autoh2o_log.zip'

In [87]:
# Clean up
os.chdir(server_path)

In [88]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    print(stats_test)   
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
      print(cf)   
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      print(cf_df)    
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

In [89]:
h2o.cluster().shutdown()

H2O session _sid_bee0 closed.
